<a href="https://colab.research.google.com/github/sands-eg/GEE_training/blob/main/MODIS_landcover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geehydro # Life saver for plotting GEE stuff with Python!
!pip install pprint
!pip install geemap

import pprint
import ee
import folium
import geehydro
import geemap
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd

  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10141 sha256=dbc7e6db6feb7649609e098db11e3ff54d2c18d942711f5369365f81ceeacb84
  Stored in directory: /root/.cache/pip/wheels/49/6c/92/019aeb37663078e6666afa9dd336e2d8df72d5683d7d357697
Successfully built geehydro
ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint
     |████████████████████████████████| 1.9 MB 36.2 MB/s 
     |████████████████████████████████| 8.6 MB 38.9 MB/s 
     |████████████████████████████████| 219 kB 50.5 MB/s 
     |████████████████████████████████| 95 kB 3.6 MB/s 
     |████████████████████████████████| 3.3 MB 38.5 MB/s 
     |████████████████████████████████| 76 kB 5.2 MB/s 
     |████████████████████████████████| 216 kB 54.7 MB/s 
     |████████████████████████████████| 130 kB 64.1 MB/s 
     |████████████████████████████████| 98 kB 7.9 MB/s 
     |████████████████████████████████| 1.2 MB 35.

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=N8tlcwKyo5wBFXYTPt2QT7J7yaMAI4O3GuLKJX1FI20&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhpPR9EUv5EAv0LEUki2ETAljXIp-yGFuMSuBlSn3fm-32OGHVkKvs

Successfully saved authorization token.


## Loading MODIS land cover data (MCD12Q1)

https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1#bands

In [ ]:
# Java code example:
# var dataset = ee.ImageCollection('MODIS/006/MCD12Q1');
# var igbpLandCover = dataset.select('LC_Type1');
# var igbpLandCoverVis = {
#   min: 1.0,
#   max: 17.0,
#   palette: [
#     '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
#     'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
#     '69fff8', 'f9ffa4', '1c0dff'
#   ],
# };
# Map.setCenter(6.746, 46.529, 6);
# Map.addLayer(igbpLandCover, igbpLandCoverVis, 'IGBP Land Cover');

In [26]:
dataset = (ee
           .ImageCollection("MODIS/006/MCD12Q1")
          #  .filterBounds(AOI)
          #  .filter(ee.Filter.lt('CLOUD_COVER', 10))
          #  .filter(ee.Filter.calendarRange(6, 8, "month"))
)
print(dataset.size().getInfo())

# IGBP classification
LandCover19 = dataset.filterDate('2015').select('LC_Type1')
LandCover01 = dataset.filterDate('2001').select('LC_Type1')

# LCCS1 land cover
# LandCover19 = dataset.filterDate('2019').select('LC_Prop1')
# LandCover01 = dataset.filterDate('2001').select('LC_Prop1')


print(LandCover01.size().getInfo())


19
1


In [25]:
print(LandCover01.getInfo())

{'type': 'ImageCollection', 'bands': [], 'id': 'MODIS/006/MCD12Q1', 'version': 1638765796098649, 'properties': {'system:visualization_0_min': '1.0', 'type_name': 'ImageCollection', 'keywords': ['landcover', 'mcd12q1', 'modis', 'nasa', 'usgs', 'yearly'], 'thumb': 'https://mw1.google.com/ges/dd/images/MODIS_006_MCD12Q1_thumb.png', 'description': '<p>The MCD12Q1 V6 product provides global land cover types at yearly intervals\n(2001-2016) derived from six different classification schemes. It is derived\nusing supervised classifications of MODIS Terra and Aqua reflectance data.\nThe supervised classifications then undergo additional post-processing that\nincorporate prior knowledge and ancillary information to further refine\nspecific classes.</p><p>Documentation:</p><ul><li><p><a href="https://lpdaac.usgs.gov/documents/101/MCD12_User_Guide_V6.pdf">User&#39;s Guide</a></p></li><li><p><a href="https://lpdaac.usgs.gov/documents/86/MCD12_ATBD.pdf">Algorithm Theoretical Basis Document (ATBD)</a

In [27]:
# parameter options based on classification choice:

# annual IGBP classification
viz_params = {
    'min': 1.0,
    'max': 17.0,
    'palette': ['05450a', '086a10', '54a708', '78d203', '009900', \
              'c6b044', 'dcd159','dade48', 'fbff13', 'b6ff05', \
              '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', \
              '69fff8', 'f9ffa4', '1c0dff'],
}

# #LCCS1 land cover layer
# viz_params = {
#     'min': 1.0,
#     'max': 43.0,
#     'palette': ['f9ffa4', '69fff8', '1c0dff', '05450a', '086a10', \
#               '54a708', '78d203','005a00', '009900', '006c00', \
#               '00d000', 'b6ff05', '98d604', 'dcd159', \
#               'f1fb58', 'fbee65'],
# }

In [28]:
# Create a default map
LON_CENTER = -60
LAT_CENTER = -10

Map = folium.Map()
Map.setCenter(LON_CENTER, LAT_CENTER, 5)
Map.addLayer(LandCover19, viz_params, '2019')
Map.addLayer(LandCover01, viz_params, '2001')
folium.LayerControl().add_to(Map)
Map